In [1]:
!pip install simpletransformers # Library ยอดนิยมในการ train transformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00
   

# Load Dataset

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
dataset = load_dataset("pythainlp/thainer-corpus-v2.2")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/591k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/201k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/207k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4379 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1475 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1472 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['words', 'ner'],
        num_rows: 4379
    })
    validation: Dataset({
        features: ['words', 'ner'],
        num_rows: 1475
    })
    test: Dataset({
        features: ['words', 'ner'],
        num_rows: 1472
    })
})

In [4]:
labels = dataset['train'].features['ner'].feature.names
labels_map = {i: labels for i, labels in enumerate(labels)}
labels

['B-PERSON',
 'I-PERSON',
 'O',
 'B-ORGANIZATION',
 'B-LOCATION',
 'I-ORGANIZATION',
 'I-LOCATION',
 'B-DATE',
 'I-DATE',
 'B-TIME',
 'I-TIME',
 'B-MONEY',
 'I-MONEY',
 'B-FACILITY',
 'I-FACILITY',
 'B-URL',
 'I-URL',
 'B-PERCENT',
 'I-PERCENT',
 'B-LEN',
 'I-LEN',
 'B-AGO',
 'I-AGO',
 'B-LAW',
 'I-LAW',
 'B-PHONE',
 'I-PHONE',
 'B-EMAIL',
 'I-EMAIL',
 'B-ZIP',
 'B-TEMPERATURE',
 'I-TEMPERATURE']

In [5]:
def transform_to_df(split):
    sentences = []
    for i, row in enumerate(split):
        words = row['words']
        labels = [labels_map[ner] for ner in row['ner']]
        sentence_id = [i] * len(words)
        sentences.extend(zip(sentence_id, words, labels))
    return pd.DataFrame(sentences, columns=["sentence_id", "words", "labels"])

# Transform train and validation splits
train_df = transform_to_df(dataset['train'])
eval_df = transform_to_df(dataset['validation'])

In [6]:
train_df.head()

,sentence_id,words,labels
0,0,ทักษิณ,B-PERSON
1,0,,I-PERSON
2,0,ชินวัตร,I-PERSON
3,1,,O
4,1,ทวีต,O


# Load Model

In [7]:
from simpletransformers.ner import NERModel, NERArgs

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 64 # Train ครั้งนึงใส่เข้าไป 64 ประโยค
model_args.num_train_epochs = 30
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True

In [9]:
# Initialize the NER model
model = NERModel(
    model_type="roberta",
    model_name="flax-community/roberta-base-thai",
    labels=labels,
    args=model_args,
    use_cuda=True
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at flax-community/roberta-base-thai and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

In [10]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:758: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 30:   0%|          | 0/69 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:782: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 5 of 30:   0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
result, model_outputs, preds_list = model.eval_model(eval_df)
result

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'eval_loss': 0.5884833534558614,
 'precision': 0.19601494396014943,
 'recall': 0.24133701318613923,
 'f1_score': 0.21632765255634964}

In [12]:
predictions, raw_outputs = model.predict(["สวัสดีครับ ผมชื่อนนท์ปวิธ เป็นตัวแทนจากบริษัท Apple"], split_on_space=False)
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/ner/ner_model.py:1643: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


[[{'ส': 'O'},
  {'ว': 'O'},
  {'ั': 'O'},
  {'ส': 'O'},
  {'ด': 'O'},
  {'ี': 'O'},
  {'ค': 'O'},
  {'ร': 'O'},
  {'ั': 'O'},
  {'บ': 'O'},
  {' ': 'O'},
  {'ผ': 'O'},
  {'ม': 'O'},
  {'ช': 'O'},
  {'ื': 'O'},
  {'่': 'O'},
  {'อ': 'O'},
  {'น': 'O'},
  {'น': 'O'},
  {'ท': 'O'},
  {'์': 'O'},
  {'ป': 'O'},
  {'ว': 'O'},
  {'ิ': 'O'},
  {'ธ': 'O'},
  {' ': 'O'},
  {'เ': 'O'},
  {'ป': 'O'},
  {'็': 'O'},
  {'น': 'O'},
  {'ต': 'O'},
  {'ั': 'O'},
  {'ว': 'O'},
  {'แ': 'O'},
  {'ท': 'O'},
  {'น': 'O'},
  {'จ': 'O'},
  {'า': 'O'},
  {'ก': 'O'},
  {'บ': 'O'},
  {'ร': 'O'},
  {'ิ': 'O'},
  {'ษ': 'O'},
  {'ั': 'O'},
  {'ท': 'O'},
  {' ': 'O'},
  {'A': 'O'},
  {'p': 'O'},
  {'p': 'O'},
  {'l': 'O'},
  {'e': 'O'}]]